In [30]:
install.packages(c('igraph', 'igraphdata'))


The downloaded binary packages are in
	/var/folders/71/32h8kjzx1d957cbl0_xhpyvr0000gn/T//RtmpHVeYKS/downloaded_packages


In [31]:
library(igraph)
library(igraphdata)

### Representing networks

#### Adjacency matrix

In [1]:
A <- rbind(c(0,1,0), c(1,0,1), c(1,0,0))
nodeNames <- c("A","B","C")
dimnames(A) <- list(nodeNames, nodeNames)
A

,A,B,C
A,0,1,0
B,1,0,1
C,1,0,0


#### Find no. of paths

In [2]:
# Each (i,j) entry of result is the number of paths of length 2 from node i to node j
A %*% A

,A,B,C
A,1,0,1
B,1,1,0
C,0,1,0


In [3]:
A %*% A %*% A %*% A # Number of path of length 4

,A,B,C
A,1,1,1
B,2,1,1
C,1,1,0


#### Edge list

In [4]:
el <- rbind(c("A","B"), c("B","A"), c("B","C"), c("C","A"))
el

A,B
B,A
B,C
C,A


#### igraph

- graph_from_literal
    - Create graph from structure given
    - By default ignore loops
    - Ignore undirected edges when the graph is directed

In [19]:
g <- graph_from_literal(A--B, B-+C, C-+A, A-+A)
g

IGRAPH 4ef8fd6 DN-- 3 2 -- 
+ attr: name (v/c)
+ edges from 4ef8fd6 (vertex names):
[1] B->C C->A

In [13]:
g <- graph_from_edgelist(el, directed=TRUE)
g

IGRAPH 0204c29 DN-- 3 4 -- 
+ attr: name (v/c)
+ edges from 0204c29 (vertex names):
[1] A->B B->A B->C C->A

In [20]:
df <- as.data.frame(el)
g <- graph_from_data_frame(df, directed=TRUE)
g

IGRAPH 6cd1924 DN-- 3 4 -- 
+ attr: name (v/c)
+ edges from 6cd1924 (vertex names):
[1] A->B B->A B->C C->A

In [22]:
g <- graph_from_adjacency_matrix(A %*% A)
g

IGRAPH 98ab6c3 DN-- 3 5 -- 
+ attr: name (v/c)
+ edges from 98ab6c3 (vertex names):
[1] A->A A->C B->A B->B C->B

### Load data to graph

In [23]:
edge_path <- 'https://raw.githubusercontent.com/dougmet/NetworksWorkshop/master/data/dolphin_edges.csv'
vertex_path <- 'https://raw.githubusercontent.com/dougmet/NetworksWorkshop/master/data/dolphin_vertices.csv'

In [24]:
dolphin_edges <- read.csv(edge_path)
dolphin_vertices <- read.csv(vertex_path)

In [26]:
head(dolphin_edges, n = 5)

,From,To
,<chr>,<chr>
1,CCL,Double
2,DN16,Feather
3,DN21,Feather
4,Beak,Fish
5,Bumper,Fish


In [27]:
head(dolphin_vertices, n = 5)

,Name,Gender
,<chr>,<chr>
1,Beak,Male
2,Beescratch,Male
3,Bumper,Male
4,CCL,Female
5,Cross,Male


In [28]:
dolphin <- graph_from_data_frame(dolphin_edges, vertices = dolphin_vertices, directed = FALSE)
dolphin

IGRAPH b3c7ddc UN-- 62 159 -- 
+ attr: name (v/c), Gender (v/c)
+ edges from b3c7ddc (vertex names):
 [1] CCL       --Double   DN16      --Feather  DN21      --Feather 
 [4] Beak      --Fish     Bumper    --Fish     DN16      --Gallatin
 [7] DN21      --Gallatin Feather   --Gallatin Beak      --Grin    
[10] CCL       --Grin     Beak      --Haecksel Grin      --Hook    
[13] Beescratch--Jet      DN21      --Jet      Feather   --Jet     
[16] Gallatin  --Jet      Haecksel  --Jonah    Beescratch--Knit    
[19] DN63      --Knit     Double    --Kringel  Hook      --Kringel 
[22] Jonah     --Kringel  Jonah     --MN105    Jet       --MN23    
+ ... omitted several edges

### igraph with USairports data

In [32]:
data('USairports')
graph_attr(USairports) # get graph properties

$name
[1] "US airports"

In [36]:
vertex_attr_names(USairports) # get vertex property names

[1] "name"     "City"     "Position"

In [42]:
vertex_attr(USairports, 'City')[1:5] # get 5 vertex city property

[1] "Bangor, ME"    "Boston, MA"    "Anchorage, AK" "New York, NY" 
[5] "Las Vegas, NV"

In [38]:
edge_attr_names(USairports) # get edge property names

[1] "Carrier"    "Departures" "Seats"      "Passengers" "Aircraft"  
[6] "Distance"

In [41]:
edge_attr(USairports, 'Carrier')[1:5] # get 5 flight (edge) Carrier property

[1] "British Airways Plc" "British Airways Plc" "British Airways Plc"
[4] "China Airlines Ltd." "China Airlines Ltd."